In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Constants
DF_LOC = '../dataset/twitter-spam/train.csv'

# 1. Base Model

In [2]:
raw_df = pd.read_csv(DF_LOC).drop('following', axis=1).drop('followers', axis=1).drop('actions', axis=1).drop('is_retweet', axis=1).drop('location', axis=1).drop('Unnamed: 7', axis=1)
raw_df.iloc[7552]

Tweet    #Virgos are usually very helpful around the ho...
Type                                          South Dakota
Name: 7552, dtype: object

In [3]:
# Preprocess type
raw_df['Type'] = raw_df['Type'].map({'Quality': 0, 'Spam': 1})
raw_df.head()

,Tweet,Type
0,Good Morning Love @LeeBrown_V,0.0
1,'@realDonaldTrump @USNavy RIP TO HEROES',1.0
2,Haven't been following the news but I understa...,0.0
3,pic.twitter.com/dy9q4ftLhZ What to do with pap...,0.0
4,#DidYouKnow ► Mahatma Gandhi made a brief visi...,0.0


In [4]:
raw_df = raw_df.drop(raw_df['Type'].loc[raw_df['Type'].isnull()].index.values)
# raw_df.isnull()

In [5]:
raw_df['Type'].loc[raw_df['Type'].isnull()]

Series([], Name: Type, dtype: float64)

In [6]:
sentences = raw_df['Tweet']

tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

In [7]:
sentences

0                           Good Morning Love  @LeeBrown_V
1                 '@realDonaldTrump @USNavy RIP TO HEROES'
2        Haven't been following the news but I understa...
3        pic.twitter.com/dy9q4ftLhZ What to do with pap...
4        #DidYouKnow ► Mahatma Gandhi made a brief visi...
                               ...                        
14894    #AllWentWrongWhen I told my hair stylist to "g...
14895    They don't have to like you, and you don't hav...
14896    #Miami Graham Nash Live at Parker Playhouse  #...
14897    @bethannhamilton is in the business of one-upp...
14898      Chasing Success  by  Space Cadetz  Listen up...
Name: Tweet, Length: 14897, dtype: object

In [7]:
# Saving via pickle
import pickle
with open('../dataset/basic-tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

ERROR: Could not find a version that satisfies the requirement pickle
ERROR: No matching distribution found for pickle


In [53]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')

print(padded[0])
print(padded.shape)

[   86   233    73 11601   158     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0]
(14897, 71)


In [95]:
# len(tokenizer.word_index)
padded[0]

array([   86,   233,    73, 11601,   158,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)

In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index), 16, input_length=71),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.BinaryAccuracy())
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 71, 16)            649152    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_24 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 129       
Total params: 670,145
Trainable params: 670,145
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.fit(padded, raw_df['Type'].values, epochs=150)

Epoch 1/150
466/466 [==============================] - 15s 26ms/step - loss: 0.2957 - binary_accuracy: 0.8642
Epoch 2/150
466/466 [==============================] - 11s 24ms/step - loss: 0.0995 - binary_accuracy: 0.9635
Epoch 3/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0405 - binary_accuracy: 0.9881
Epoch 4/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0185 - binary_accuracy: 0.9944
Epoch 5/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0081 - binary_accuracy: 0.9974
Epoch 6/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0067 - binary_accuracy: 0.9987
Epoch 7/150
466/466 [==============================] - 10s 22ms/step - loss: 0.0039 - binary_accuracy: 0.9987
Epoch 8/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0061 - binary_accuracy: 0.9979
Epoch 9/150
466/466 [==============================] - 11s 23ms/step - loss: 0.0024 - binary_accuracy: 0.9994
Epoch 10/1

In [79]:
model.save('../model/twitter-spam')

2021-10-27 19:32:32.794185: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../model/twitter-spam/assets


INFO:tensorflow:Assets written to: ../model/twitter-spam/assets


In [81]:
# Testing
squid_df = pd.read_json('../dataset/squid-tweets/squid_tweets.json')
squid_df = squid_df.rename(columns={0: "tweets"})
squid_df.head()

,tweets
0,RT @sweetshrubs_: More skts x squid game doodl...
1,@milesSI Squid Game any good?
2,I've just watched episode S01 | E02 of Squid G...
3,RT @binseolovely: SQUID GAME (#STRAYKIDS VER.)...
4,RT @a_leesha1: We kinda already have a real li...


In [127]:
input_pred = tokenizer.texts_to_sequences(squid_df['tweets'])
# print(input_pred, end='\n===\n')
input_pad = tf.keras.preprocessing.sequence.pad_sequences(input_pred, padding='post', maxlen=71)
# print(input_pad, end='\n===\n')
is_spam = model.predict(input_pad)

squid_df['is_spam'] = [True if i >= 0.5 else False for i in is_spam]

# for i in squid_df['tweets']:
#     input_pred = tokenizer.texts_to_sequences(i)
#     print(input_pred, end='\n===\n')
#     input_pad = tf.keras.preprocessing.sequence.pad_sequences([input_pred], padding='post', maxlen=71)
#     print(input_pad, end='\n===\n')
#     model.predict(input_pad)

In [128]:
squid_df.head()

,tweets,is_spam
0,RT @sweetshrubs_: More skts x squid game doodl...,True
1,@milesSI Squid Game any good?,False
2,I've just watched episode S01 | E02 of Squid G...,True
3,RT @binseolovely: SQUID GAME (#STRAYKIDS VER.)...,True
4,RT @a_leesha1: We kinda already have a real li...,False


In [130]:
!pip install openpyxl
squid_df.to_excel('../dataset/squid-tweets/squid-dataset.xlsx')

     |████████████████████████████████| 242 kB 527 kB/s 


In [ ]:
squid_df['spam'] = model.predict()